In [3]:
# import sys 
# !conda install -c conda-forge --prefix {sys.prefix} -y findspark
# !conda install -c conda-forge --prefix {sys.prefix} -y requests
# !conda install -c conda-forge --prefix {sys.prefix} -y json
# !conda install -c conda-forge --prefix {sys.prefix} -y pyspark

### Findspark
findspark 초기화

In [1]:
import findspark
# findspark.add_packages('mysql:mysql-connector-java-8.0.28')
findspark.init()

### PySpark 관련 패키지 import

In [2]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.types import *
from pyspark.sql.functions import *

### api 처리 패키지 import

In [3]:
from datetime import datetime, timedelta
import requests
import json

### General 패키지 import

In [4]:
import configparser
from pathlib import Path

### Local SparkCluster connected session

In [5]:
spark = SparkSession \
        .builder \
        .master('local') \
        .appName('tempera_obs') \
        .config("spark.driver.extraClassPath", "/Users/mdgome/Downloads/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar") \
        .getOrCreate()
#        .master('spark://hadoop01:7077') \
       # .config("spark.driver.extraClassPath", "/hadoop/jupyter_dir/jar/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar") \
            # .config("spark.driver.extraClassPath", "hdfs:///spark3-jars/mysql-connector-java-8.0.28.jar") \

22/03/09 01:23:49 WARN Utils: Your hostname, Jungminui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.19 instead (on interface en0)
22/03/09 01:23:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 01:23:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### DB Server Configure & API Service Key

In [6]:
config = configparser.ConfigParser()
config.read(Path("./config/config.ini"),encoding='utf-8')
# config.read(os.getcwd()+os.sep+'config'+os.sep+'config.ini',encoding='utf-8')

user = config['dev_mysql']['user']
password = config['dev_mysql']['password']
host = config['dev_mysql']['host']
port = config['dev_mysql']['port']
dbname = config['dev_mysql']['dbname']
url = config['dev_mysql']['url'].format(host=host,port=port,dbname=dbname)
service_key=config['api']['service_key']
# dbtable = config['dev_mysql']['dbtable']

#### 관측소 정보 table 불러옴

In [7]:
# obs_post_data = pd.read_csv("./관측소 정보.csv")
obs_post_data= spark.read.format('jdbc').options(
    url=url,
    driver='com.mysql.jdbc.Driver',
    dbtable='obs_info',
    user=user,
    password=password).load()
obs_post_data.show(5)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+----------+---------+----------+----------------------------------+-----------+-------------+
| data_type|  obs_lat|   obs_lon|                        obs_object|obs_post_id|obs_post_name|
+----------+---------+----------+----------------------------------+-----------+-------------+
|조위관측소|35.024178|128.810933|조위,수온,염분,기온,기압,풍속,풍향|    DT_0063|       가덕도|
|조위관측소|37.731944|126.522222|                         조위,기압|    DT_0032|     강화대교|
|조위관측소|34.028333|127.308889|조위,수온,염분,기온,기압,풍속,풍향|    DT_0031|       거문도|
|조위관측소|34.801389|128.699167|조위,수온,염분,기온,기압,풍속,풍향|    DT_0029|       거제도|
|조위관측소|37.560833|126.601111|          조위,기온,기압,풍속,풍향|    DT_0058|       경인항|
+----------+---------+----------+----------------------------------+-----------+-------------+
only showing top 5 rows



In [ ]:
# test_df = obs_post_data.select(split(col("obs_object"), ",").alias("obs_object_array")).drop("obs_object")
# obs_post_data.createOrReplaceTempView("obs_post")
# obs_post_data = spark.sql("select *, SPLIT(obs_object,',') as obs_object_array from obs_post order").drop("obs_object")

## 관측소 별 호출 가능한 API 정리

In [8]:
obs_info_df = obs_post_data.select(obs_post_data.obs_post_id,obs_post_data.obs_post_name,obs_post_data.obs_lat,obs_post_data.obs_lon,split(col("obs_object"), ",").alias("obs_object_array"),obs_post_data.data_type).drop("obs_object")
obs_info_df = obs_info_df.sort(col("obs_object_array"),col("obs_post_id"))
obs_info_df.show(5)

+-----------+-------------+---------+----------+----------------------------+------------+
|obs_post_id|obs_post_name|  obs_lat|   obs_lon|            obs_object_array|   data_type|
+-----------+-------------+---------+----------+----------------------------+------------+
|    DT_0065|       덕적도|37.226333|126.156556|    [기온, 기압, 풍속, 풍향]|  조위관측소|
|    HB_0002|  한수원_고리|  35.3185|129.314722|[수온, 기온, 기압, 풍속, ...|해양관측부이|
|    HB_0008|  한수원_덕천|     37.1|129.404167|[수온, 기온, 기압, 풍속, ...|해양관측부이|
|    TW_0079|     상왕등도|35.652472| 126.19425|[수온, 기온, 기압, 풍속, ...|해양관측부이|
|    KG_0024|     대한해협|   34.919| 129.12125|[수온, 기온, 기압, 풍속, ...|해양관측부이|
+-----------+-------------+---------+----------+----------------------------+------------+
only showing top 5 rows



In [11]:
for row in obs_info_df.rdd.collect():
    print(row['obs_post_id'], row['obs_object_array'], row['data_type'])
    # print(type(row['obs_object']))

DT_0065 ['기온', '기압', '풍속', '풍향'] 조위관측소
HB_0002 ['수온', '기온', '기압', '풍속', '풍향', '유속', '유향'] 해양관측부이
HB_0008 ['수온', '기온', '기압', '풍속', '풍향', '유속', '유향'] 해양관측부이
TW_0079 ['수온', '기온', '기압', '풍속', '풍향', '유속', '유향'] 해양관측부이
KG_0024 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
KG_0025 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
KG_0101 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0062 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0069 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0075 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0080 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0081 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0089 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0090 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0091 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0092 ['수온', '기온', '기압', '풍속', '풍향', '파고', '유속', '유향'] 해양관측부이
TW_0093 ['수온', '기온'

In [14]:
obs_post_list = obs_post_data.select('obs_post_id').rdd.flatMap(lambda x: x).collect()
# print(list(obs_post_list)) # 디버깅 용

2022-03-03 08:03:56,620 WARN scheduler.TaskSetManager: Lost task 13.0 in stage 8.0 (TID 22) (hadoop02 executor 1): java.io.IOException: Cannot run program "/hadoop/anaconda3/envs/jungmin/bin/python": error=2, 그런 파일이나 디렉터리가 없습니다
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apa

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 8.0 failed 4 times, most recent failure: Lost task 13.3 in stage 8.0 (TID 26) (hadoop02 executor 1): java.io.IOException: Cannot run program "/hadoop/anaconda3/envs/jungmin/bin/python": error=2, 그런 파일이나 디렉터리가 없습니다
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RD

#### 전날 데이터 불러오기 위한 관련 코드

In [104]:
yesterday = datetime.today() - timedelta(days = 1 )
yesterday = yesterday.strftime('%Y%m%d')
yesterday

'20220301'

In [106]:
# 기온 api
url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 기압 api
url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 풍속 api
url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 염분 api
url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 조위 실측/예측 api
url4="http://www.khoa.go.kr/api/oceangrid/tideCurPre/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 수온 api
url5="http://www.khoa.go.kr/api/oceangrid/tideObsTemp/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"

#### API data 가져옴

In [107]:
response = requests.get(url).json()
response1 = requests.get(url1).json()
response2 = requests.get(url2).json()
response3 = requests.get(url3).json()
response4 = requests.get(url4).json()
response5 = requests.get(url5).json()

#### 가져온 data spark dataframe 으로 변환

In [108]:
df = spark.createDataFrame(response["result"]["data"])
df1 = spark.createDataFrame(response1["result"]["data"])
df2 = spark.createDataFrame(response2["result"]["data"])
df3 = spark.createDataFrame(response3["result"]["data"])
df4 = spark.createDataFrame(response4["result"]["data"])
df5 = spark.createDataFrame(response5["result"]["data"])

#### df1, df2, df3, df5 하나의 dataframe 으로 합침

In [122]:
output_df = df.join(df1,'record_time')#.select(df1.record_time, df1.air_pres, df.air_temp)
output_df = output_df.join(df2, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,df2.wind_dir,df2.wind_speed)
output_df = output_df.join(df3, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
output_df = output_df.join(df5, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
# output_df = output_df.join(df4, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,output_df.salinity,df4.pre_value,df4.real_value)

In [113]:
output_df.printSchema()

root
 |-- record_time: string (nullable = true)
 |-- air_temp: string (nullable = true)
 |-- air_pres: string (nullable = true)
 |-- wind_dir: string (nullable = true)
 |-- wind_speed: string (nullable = true)
 |-- salinity: string (nullable = true)
 |-- water_temp: string (nullable = true)



In [114]:
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+----------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|
+-------------------+--------+--------+--------+----------+--------+----------+
|2022-03-01 05:45:00|    13.6|  1013.4|   236.0|       5.4|    32.6|      13.3|
|2022-03-01 07:27:00|    12.7|  1014.6|   298.0|       7.4|    32.5|      13.3|
|2022-03-01 09:29:00|    11.5|  1014.9|   286.0|       5.4|    32.5|      13.4|
|2022-03-01 12:58:00|    10.6|  1014.7|   282.0|       4.5|    32.5|      13.3|
|2022-03-01 13:47:00|    11.3|  1014.2|   293.0|       5.0|    32.5|      13.2|
+-------------------+--------+--------+--------+----------+--------+----------+
only showing top 5 rows



#### 모든 data type 이 string으로 되어 있어 data type 변경

In [123]:
output_df = output_df.withColumn('air_temp', output_df['air_temp'].cast(DoubleType()))\
    .withColumn('air_pres', output_df['air_pres'].cast(DoubleType()))\
    .withColumn('wind_dir', output_df['wind_dir'].cast(DoubleType()))\
    .withColumn('record_time', output_df['record_time'].cast("timestamp"))\
    .withColumn('salinity', output_df['salinity'].cast(DoubleType()))
    # .withColumn('pre_value', df4['pre_value'].cast(IntegerType())) \
    # .withColumn('real_value', df4['real_value'].cast(IntegerType()))

In [124]:
output_df.printSchema()

root
 |-- record_time: timestamp (nullable = true)
 |-- air_temp: double (nullable = true)
 |-- air_pres: double (nullable = true)
 |-- wind_dir: double (nullable = true)
 |-- wind_speed: string (nullable = true)
 |-- salinity: double (nullable = true)
 |-- water_temp: string (nullable = true)



In [125]:
df4 = df4.withColumn('pre_value', df4['pre_value'].cast(IntegerType())) \
    .withColumn('record_time', df4['record_time'].cast("timestamp"))\
    .withColumn('real_value', df4['real_value'].cast(IntegerType()))

In [126]:
df4.printSchema()

root
 |-- pre_value: integer (nullable = true)
 |-- real_value: integer (nullable = true)
 |-- record_time: timestamp (nullable = true)



In [127]:
df4.show(5)

+---------+----------+-------------------+
|pre_value|real_value|        record_time|
+---------+----------+-------------------+
|      134|       160|2022-03-01 00:00:00|
|       92|       121|2022-03-01 01:00:00|
|       53|        81|2022-03-01 02:00:00|
|       23|        51|2022-03-01 03:00:00|
|       14|        39|2022-03-01 04:00:00|
+---------+----------+-------------------+
only showing top 5 rows



In [129]:
output_df = output_df.sort(asc("record_time"))
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+----------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|
+-------------------+--------+--------+--------+----------+--------+----------+
|2022-03-01 00:00:00|    10.9|  1016.1|   278.0|       0.3|    32.6|      13.2|
|2022-03-01 00:01:00|    10.9|  1016.0|   278.0|       0.4|    32.6|      13.2|
|2022-03-01 00:02:00|    10.8|  1016.0|   278.0|       0.7|    32.6|      13.2|
|2022-03-01 00:03:00|    10.7|  1016.0|   210.0|       1.0|    32.6|      13.2|
|2022-03-01 00:04:00|    10.7|  1016.1|   160.0|       0.5|    32.6|      13.2|
+-------------------+--------+--------+--------+----------+--------+----------+
only showing top 5 rows



In [103]:
union_df = output_df.unionByName(df4, allowMissingColumns=True)
union_df = union_df.sort(asc("record_time"))
union_df.show(5)

+-------------------+--------+--------+--------+----------+--------+----------+---------+----------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|pre_value|real_value|
+-------------------+--------+--------+--------+----------+--------+----------+---------+----------+
|2022-03-01 00:00:00|    10.9|  1016.1|   278.0|       0.3|    32.6|      13.2|     null|      null|
|2022-03-01 00:00:00|    null|    null|    null|      null|    null|      null|      134|       160|
|2022-03-01 00:01:00|    10.9|  1016.0|   278.0|       0.4|    32.6|      13.2|     null|      null|
|2022-03-01 00:02:00|    10.8|  1016.0|   278.0|       0.7|    32.6|      13.2|     null|      null|
|2022-03-01 00:03:00|    10.7|  1016.0|   210.0|       1.0|    32.6|      13.2|     null|      null|
+-------------------+--------+--------+--------+----------+--------+----------+---------+----------+
only showing top 5 rows



In [38]:
output_df = output_df.join(df4, 'record_time').select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,output_df.salinity,df4.pre_value,df4.real_value)
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+---------+----------+
|        record_time|air_pres|air_temp|wind_dir|wind_speed|salinity|pre_value|real_value|
+-------------------+--------+--------+--------+----------+--------+---------+----------+
|2022-03-01 17:00:00|  1016.0|     9.5|   312.0|       5.5|    32.7|       60|        70|
|2022-03-01 01:00:00|  1015.6|    11.2|   117.0|       1.7|    32.7|       92|       121|
|2022-03-01 04:00:00|  1013.0|    12.2|   276.0|       1.3|    32.6|       14|        39|
|2022-03-01 18:00:00|  1017.0|     8.4|   321.0|       5.6|    32.7|       84|        90|
|2022-03-01 22:00:00|  1019.8|     7.4|   284.0|       4.2|    32.6|      214|       222|
+-------------------+--------+--------+--------+----------+--------+---------+----------+
only showing top 5 rows



#### 합쳐진 dataframe sorting

In [39]:
output_df = output_df.sort(asc("record_time"))
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+---------+----------+
|        record_time|air_pres|air_temp|wind_dir|wind_speed|salinity|pre_value|real_value|
+-------------------+--------+--------+--------+----------+--------+---------+----------+
|2022-03-01 00:00:00|  1016.1|    10.9|   278.0|       0.3|    32.6|      134|       160|
|2022-03-01 01:00:00|  1015.6|    11.2|   117.0|       1.7|    32.7|       92|       121|
|2022-03-01 02:00:00|  1014.6|    12.2|   104.0|       1.7|    32.6|       53|        81|
|2022-03-01 03:00:00|  1013.5|    12.1|   140.0|       0.5|    32.6|       23|        51|
|2022-03-01 04:00:00|  1013.0|    12.2|   276.0|       1.3|    32.6|       14|        39|
+-------------------+--------+--------+--------+----------+--------+---------+----------+
only showing top 5 rows



#### DB Table 에서 해당 데이터가 어디 관측소에서 수집 됐는지 알기 위한 value 추가

In [40]:
output_df = output_df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [41]:
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+---------+----------+-------+
|        record_time|air_pres|air_temp|wind_dir|wind_speed|salinity|pre_value|real_value|     id|
+-------------------+--------+--------+--------+----------+--------+---------+----------+-------+
|2022-03-01 00:00:00|  1016.1|    10.9|   278.0|       0.3|    32.6|      134|       160|DT_0004|
|2022-03-01 01:00:00|  1015.6|    11.2|   117.0|       1.7|    32.7|       92|       121|DT_0004|
|2022-03-01 02:00:00|  1014.6|    12.2|   104.0|       1.7|    32.6|       53|        81|DT_0004|
|2022-03-01 03:00:00|  1013.5|    12.1|   140.0|       0.5|    32.6|       23|        51|DT_0004|
|2022-03-01 04:00:00|  1013.0|    12.2|   276.0|       1.3|    32.6|       14|        39|DT_0004|
+-------------------+--------+--------+--------+----------+--------+---------+----------+-------+
only showing top 5 rows



#### DB data insert

In [30]:
output_df.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='DT_0004',
    user='mdgome',
    password='Rlawjdals1!').mode('append').save()

In [31]:
df4.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='DT_0004_tideCurPre',
    user='mdgome',
    password='Rlawjdals1!').mode('append').save()

In [33]:
spark.stop()